# SpaceX Launch Sites Analysis with Folium

Este notebook cobre as três tarefas principais:

- **Task 1**: Marcar todos os launch sites no mapa
- **Task 2**: Marcar os lançamentos (sucesso/falha)
- **Task 3**: Calcular distâncias até proximidades (coastline, highway, city)

Todos os mapas são interativos e podem ser salvos como HTML.

In [ ]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import radians, sin, cos, sqrt, atan2

# Função de distância geodésica (Haversine)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2-lat1, lon2-lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1-a))


In [ ]:
# Carregar dataset original
spacex_df = pd.read_csv("spacex_launch_geo (1).csv")
spacex_df.head()


## Task 1: Mark all launch sites on a map

In [ ]:
launch_sites_df = spacex_df[['Launch Site','Lat','Long']].drop_duplicates().reset_index(drop=True)

site_map = folium.Map(location=[30,-95], zoom_start=4, tiles="OpenStreetMap")

for _, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    folium.Circle(coord, radius=1000, color='#d35400', fill=True).add_child(
        folium.Popup(row['Launch Site'])
    ).add_to(site_map)
    folium.map.Marker(coord,
        icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{row["Launch Site"]}</b></div>')
    ).add_to(site_map)

site_map.save("task1_launch_sites.html")
site_map


## Task 2: Mark the success/failed launches

In [ ]:
# Criar coluna de cores
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x==1 else 'red')

launch_map = folium.Map(location=[30,-95], zoom_start=4)
marker_cluster = MarkerCluster().add_to(launch_map)

for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"{row['Launch Site']} | Outcome: {'Success' if row['class']==1 else 'Failure'}",
        icon=folium.Icon(color='white', icon_color=row['marker_color'], icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

launch_map.save("task2_launch_outcomes.html")
launch_map


## Task 3: Calculate distances between a launch site and proximities

In [ ]:
launch_sites = {
    "CCAFS LC-40": (28.562302, -80.577356),
    "CCAFS SLC-40": (28.563197, -80.576820),
    "KSC LC-39A": (28.573255, -80.646895),
    "VAFB SLC-4E": (34.632834, -120.610745)
}

points_of_interest = {
    "Coastline Cape": (28.56367, -80.57163),
    "Orlando City": (28.5383, -81.3792),
    "Highway KSC": (28.572, -80.799),
    "Lompoc City": (34.6391, -120.4579)
}

distance_map = folium.Map(location=[30, -95], zoom_start=4)

for site_name, (lat, lon) in launch_sites.items():
    folium.Marker([lat, lon], popup=site_name,
                  icon=folium.Icon(color="blue", icon="rocket", prefix="fa")).add_to(distance_map)
    for poi_name, (plat, plon) in points_of_interest.items():
        dist = calculate_distance(lat, lon, plat, plon)
        folium.Marker([plat, plon], popup=poi_name,
                      icon=folium.Icon(color="red", icon="info-sign")).add_to(distance_map)
        folium.PolyLine([[lat, lon], [plat, plon]], color="green",
                        tooltip=f"{site_name} → {poi_name}: {dist:.2f} km").add_to(distance_map)

distance_map.save("task3_launch_site_distances.html")
distance_map


### Observações

- Todos os launch sites estão próximos da costa → isso facilita lançamentos seguros sobre o oceano.
- Há ferrovias e highways bem próximas de alguns sites, facilitando logística.
- Os launch sites mantêm distância de cidades grandes → minimiza riscos em caso de falhas.
- Proximidade ao Equador ajuda em eficiência de lançamentos orbitais.
